In [4]:
#import dependency
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import requests
#Import password for AWS database
from config import password

# **Connect to AWS Database using spark**

In [8]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' 不是內部或外部命令、可執行的程式或批次檔。
系統找不到指定的路徑。
'wget' 不是內部或外部命令、可執行的程式或批次檔。
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

'wget' 不是內部或外部命令、可執行的程式或批次檔。


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [29]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ce3jlxk8s6lq.us-east-2.rds.amazonaws.com:5432/house_prices_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

NameError: ignored

In [ ]:
pushdown_query = "(select * from public.\"HOUSE_PRICES\") house"
df2 = spark.read.jdbc(url=jdbc_url, table=pushdown_query, properties=config)
df2.head()

Row(HOUSE_ID=1462, MSSUBCLASS_ID=20, MSZONING_ID='RL', LOTFRONTAGE=81, LOTAREA=14267, STREET_ID='Pave', ALLEY_ID='NA', LOTSHAPE_ID='IR1', LANDCONTOUR_ID='Lvl', UTILITIES_ID='AllPub', LOTCONFIG_ID='Corner', LANDSLOPE_ID='Gtl', NEIGHBORHOOD_ID='NAmes', CONDITION1_ID='Norm', CONDITION2_ID='Norm', BLDGTYPE_ID='1Fam', HOUSESTYLE_ID='1Story', OVERALLQUAL_ID=6, OVERALLCOND_ID=6, YEARBUILT=1958, YEARREMODADD=1958, ROOFSTYLE_ID='Hip', ROOFMATL_ID='CompShg', EXTERIOR1ST_ID='Wd Sdng', EXTERIOR2ND_ID='Wd Sdng', MASVNRTYPE_ID='BrkFace', MASVNRAREA=108, EXTERQUAL_ID='TA', EXTERCOND_ID='TA', FOUNDATION_ID='CBlock', BSMTQUAL_ID='TA', BSMTCOND_ID='TA', BSMTEXPOSURE_ID='No', BSMTFINTYPE1_ID='ALQ', BSMTFINSF1=923, BSMTFINTYPE2_ID='Unf', BSMTFINSF2=0, BSMTUNFSF=406, TOTALBSMTSF=1329, HEATING_ID='GasA', HEATINGQC_ID='TA', CENTRALAIR='Y', ELECTRICAL_ID='SBrkr', 1STFLRSF=1329, 2NDFLRSF=0, LOWQUALFINSF=0, GRLIVAREA=1329, BSMTFULLBATH=0, BSMTHALFBATH=0, FULLBATH=1, HALFBATH=1, BEDROOMABVGR=3, KITCHENABVGR=1, K

In [ ]:
print(type(df2))

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
df_train = pd.DataFrame(df2) 

ValueError: ignored

# **Step 1: Data Exploration**
* Check the dataset
* check the missing values and duplicates
* check the columns
* check the types
* check the shape

In [ ]:
missing = df2.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

AttributeError: ignored

In [ ]:
train_df.columns.tolist()

NameError: ignored

In [ ]:
train_df.shape

NameError: ignored

In [ ]:
train_df.describe()

NameError: ignored

In [ ]:
feature = df2.drop(columns=['SalePrice'])
target = df2['SalePrice']

print(feature)
print(target)

TypeError: ignored

In [ ]:
feature.dtypes

NameError: ignored

In [ ]:
target.dtypes


NameError: ignored

**Data Explorarion Summary:**
* There are multiple types of features, both categorical and numerical
* Tratget is numerical
* There are missing values in the dataset



# **Step 2: Feature Selection:**
* USe heatmap to identify features that are most correlated to the Sales Price
* check histogram and see if they are normally distributed in order to be used in linear regression model


In [ ]:
corrmat = train_df.corr()
f, ax = plt.subplots(figsize=(20, 9))
sns.heatmap(corrmat, vmax=0.8, square=True)

NameError: ignored

In [ ]:
#histogram
sns.distplot(train_df['SalePrice']);


NameError: ignored

# **Features Selected are:**
* OverallQual
* YearBuilt
* ToatlBsmtSF
* 1stFlrSF
* GrLiveArea
* FullBath
* TotRmsAbvGrd
* GarageCars
* GarageArea


# **Step 3: Data cleaning:**
* remove NA/replace with the mode of their respective columns
* remove duplicate
* transform the skewed numeric features by taking log(feature + 1) -->make features more normally distributed
* Create Dummy variables for the categorical features
* create new feature to combine features that are in the same category

# **Step 4: Multiple Linear Regression Model: Predict house prices based on the selected features**
* create a list that contains the 7 selected features 
* Split the data into input (X) and output (y).
* Create an instance of the model with model = LinearRegression().
* Train the model with the dataset with model.fit(X,y).
* Create predictions with y_pred = model.predict(X).

In [ ]:
#Machine Learnining
cols = ['OverallQual','GrLivArea', 'GarageCars','TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']
x = trained_df[cols].values
y = trained_df['SalePrice'].values
x_scaled = preprocessing.StandardScaler().fit_transform(x)
y_scaled = preprocessing.StandardScaler().fit_transform(y.reshape(-1,1))
X_train,X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.33, random_state=42)

clf = RandomForestRegressor(n_estimators=400)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(y_pred)

model = LinearRegression()
# We will be using the columns from the dataset excluding Sales Price to output the SalesPrice
model.fit(X, y)

y_pred = model.predict(X)

print(y_pred.shape)

plt.scatter(X, y)
plt.plot(X, y_pred, color='red')
plt.show()

NameError: ignored


# **Step 5: Statistical Summary to check the model accuracy**

In [ ]:
print(model.coef_)
print(model.intercept_)

NameError: ignored